In [ ]:
import numpy as np
import pandas as pd
import joblib


df = pd.read_excel('/content/model_label_T.xlsx')
print(df)

In [ ]:
import numpy as np

play_df = df["놀이 이름"].values.tolist()
play_list = [play_df for play_df in df['놀이 이름']]

print(play_list)

info_list = df[['개월 수', '인지', '정서', '언어', '운동', '감각', '야외 활동(0=false/1=true)', '교구 활용(0=false/1=true)']].values.tolist()


In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np

# 입력 데이터와 라벨 데이터 준비
X = info_list
y = play_df

# 라벨 인코딩
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

X_train = np.array(X_train)
X_test = np.array(X_test)

# TensorFlow 모델 정의
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')  # 출력 레이어
])

# 모델 컴파일
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 모델 학습
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=2)

# 모델 평가
loss, accuracy = model.evaluate(X_test, y_test)
print("Accuracy:", accuracy)

# 추천 함수 정의
def recommend_play(child_info):
    # 모델을 통해 추천을 예측
    recommended_play_encoded = model.predict(np.array([child_info]))
    # 가장 높은 확률을 가진 클래스를 선택하여 디코딩
    recommended_play_index = np.argmax(recommended_play_encoded)
    recommended_play = label_encoder.inverse_transform([recommended_play_index])
    return recommended_play[0]

# 아이에게 가장 적합한 놀이 추천
child_info = [0, 10, 90, 56, 25, 30, 0, 0]  # 아이의 정보 입력
recommended_play = recommend_play(child_info)
print("Recommended Play:", recommended_play)


joblib.dump(model, 'best_model.pkl')